In [ ]:
!pip install geojson
!pip install rasterio
!pip install pytorch_lightning

In [ ]:
import os
import gdal
from matplotlib import pyplot as plt
import numpy as np
import cv2
import pandas as pd
import rasterio
from rasterio.mask import mask as rasterio_mask
import geojson
import random
import argparse
from osgeo import gdal
from osgeo.gdalconst import GDT_Float32
import sys
import math
import itertools
import time
import random


from sklearn import metrics
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OneVsRestClassifier

from torch.utils.data import Dataset, DataLoader
import torchvision
import torch
from torch import nn
from torch.nn import functional as F
from torchvision import models

import pytorch_lightning as pl

In [ ]:
def get_img(snapshot):
    plt.figure(figsize=(7, 7))
    plt.axis("off")
    plt.imshow(cv2.merge([snapshot[3], snapshot[2], snapshot[1]]))

In [ ]:
def create_df(field_g, field_class, max_canal, num_def_field):
    df_def_g = pd.DataFrame(columns=[i for i in range(max_canal)]+['class', 'id'])
    for field_i in field_g:
        df_def_i = pd.DataFrame(field_i.reshape(max_canal, -1).transpose())
        df_def_i['id'] = num_def_field
        num_def_field += 1
        #df_def_i = df_def_i.loc[(df_def_i[2]>0)&(df_def_i[3]>0)]
        df_def_g = pd.concat([df_def_g, df_def_i], ignore_index=True)
    df_def_g['class'] = field_class
    return df_def_g, num_def_field

In [ ]:
def get_id_test(df_def, part_test, max_shuffle=30):
    df_def_count = df_def.groupby('id', as_index=False).count()[['id', 'class']]
    max_count = df_def_count['class'].sum()
    min_count = 1
    min_id_test = []
    i_def = 0
    while i_def<max_shuffle and abs(min_count-part_test)>0.01:
        sum_count = 0
        id_test = []
        for index_row, df_count_row in df_def_count.sample(frac=1).iterrows():
            sum_count += df_count_row['class']
            id_test.append(df_count_row['id'])
            cur_count = sum_count/max_count
            if cur_count>=part_test-0.02:
                if abs(cur_count-part_test)<abs(min_count-part_test):
                    min_count = cur_count
                    min_id_test = id_test
                i_def += 1
                break
    if min_count>part_test*2 or part_test-0.96>0:
        min_id_test = []
    return min_id_test

In [ ]:
def create_prepared_df(list_df_def, list_data_def, remove_cloud=True, with_class=False):
    df_def = pd.DataFrame()
    for data_def, df_i_def in zip(list_data_def, list_df_def):
        df_i_def = df_i_def.rename(columns={x:str(x)+'_'+data_def for x in (df_i_def.columns[:-2] if with_class else df_i_def.columns)})
        df_def = df_def.join(df_i_def[(df_i_def.columns[:-2] if with_class else df_i_def.columns)], how="outer")
    if with_class:
        df_def['class'] = df_i_def['class']
    if remove_cloud:
        for col in df_def.columns[:-1]:
            if col.split('_')[0]=='12':
                df_def = df_def.loc[df_def[col]<1].reset_index(drop=True)
    return df_def

In [ ]:
def kill_max_class(df_def):
    df_count = df_def.groupby('class', as_index=False).count()
    df_count.index = df_count['class']
    class_median = int(df_count[df_count.columns[1]].median())
    max_class = df_count[df_count.columns[1]].idxmax()
    df_def_max = df_def.loc[df_def['class']==max_class]
    df_def_max['ind'] = df_def_max.index
    df_def_max = df_def_max.loc[df_def_max['ind'].isin(random.sample(list(df_def_max.index), class_median))]
    df_def = pd.concat([df_def.loc[df_def['class']!=max_class], df_def_max[df_def_max.columns[:-1]]], ignore_index=True).sort_values(['class']).reset_index(drop=True)
    return df_def

In [ ]:
def write_geotiff(raster_input, raster_output, predictions):
    """
    Function to replace values in geotiff upper then threshold by thresholf  🙂
    
    Input : GeoTIFF, threshold_value
    
    Output : GeoTIFF 
    
    """    
    in_data, out_data = None, None

    # open input raster
    in_data = gdal.Open(raster_input)
    if in_data is None:
        print ('Unable to open %s' % raster_input)
        return None

    # read in data from first band of input raster
    band1 = in_data.GetRasterBand(1)
    rows = in_data.RasterYSize
    cols = in_data.RasterXSize
    vals = band1.ReadAsArray(0, 0, cols, rows)

    driver = in_data.GetDriver()
    out_data = driver.Create(raster_output, cols, rows, 1, GDT_Float32)

    dem_data = np.array(predictions)
    out_band = out_data.GetRasterBand(1)
    out_band.WriteArray(dem_data)
    out_band.FlushCache()
    out_band.SetNoDataValue(-32767.)

    out_data.SetGeoTransform(in_data.GetGeoTransform())
    out_data.SetProjection(in_data.GetProjection())
    del out_data
    return raster_output

In [ ]:
path = "/Forest/data/"
name_field = 'Bratsk'
resolution = '10'
folder_out = '10'
#folder_out = '10_only_forest'

path_field = path + name_field + '/' + resolution + '/'
path_field_year = [path_field+f+'/' for f in os.listdir(path_field) if f!='Masks']
path_snapshots = [field_year+snapshot for field_year in path_field_year for snapshot in os.listdir(field_year)]
path_masks = path + name_field + '/' + 'Masks/In/' + os.listdir(path + name_field + '/' + 'Masks/In/')[0]
path_out_masks = path + name_field + '/' + 'Masks/Out/' + folder_out + '/'

if not os.path.exists(path_out_masks):
    os.mkdir(path_out_masks)

In [ ]:
remove_autumn_months = True
if remove_autumn_months:
    while [path_snapshot for path_snapshot in path_snapshots if (int(path_snapshot.split('/')[-1].split('.')[0][6:7])<6 or int(path_snapshot.split('/')[-1].split('.')[0][6:7])>8)]:
        for path_snapshot in [path_snapshot for path_snapshot in path_snapshots if (int(path_snapshot.split('/')[-1].split('.')[0][6:7])<6 or int(path_snapshot.split('/')[-1].split('.')[0][6:7])>8)]:
            path_snapshots.remove(path_snapshot)

In [ ]:
count_cloud = float("inf")
for path_snapshot in path_snapshots:
    if path_snapshot[-9]=='7' or path_snapshot[-9]=='8':
        #print(path_snapshot)
        field = np.array(gdal.Open(path_snapshot).ReadAsArray())
        df_i = pd.DataFrame(field.reshape(13, -1).transpose())
        if len(df_i.loc[df_i[12]>0])<count_cloud:
            count_cloud = len(df_i.loc[df_i[12]>0])
            path_min_cloud = path_snapshot
        if count_cloud==0:
            print(path_snapshot)
            #break

In [ ]:
field = np.array(gdal.Open(path_min_cloud).ReadAsArray())
get_img(field)

with open(path_masks) as f:
    gj = geojson.load(f)
features = [feature["geometry"] for feature in gj['features']]

with rasterio.open(path_min_cloud) as src:
    out_image, _ = rasterio_mask(src, features, crop=True)

get_img(out_image)

In [ ]:
def convert_to_geotiff(raster_input, raster_output, values_to_write):
    """
    Save GPP resul geotiff GeoTiff

    input: path to geotiff with fPAR 
    output: path to new geotiff with GPP 
    values_to_write: np.array with dimensions like input geotiff file

    """
    in_data, out_data = None, None
    in_data = gdal.Open(raster_input)
    if in_data is None:
        print ('Unable to open %s' % raster_input)

    # read in data from first band of input raster
    band1 = in_data.GetRasterBand(1)
    rows = in_data.RasterYSize
    cols = in_data.RasterXSize
    vals = band1.ReadAsArray(0, 0, cols, rows)

    driver = in_data.GetDriver()

    num_of_layers = values_to_write.shape[0]

    out_data = driver.Create(raster_output, cols, rows, num_of_layers, GDT_Float32)

    # Save data
    for i in range(num_of_layers):
        out_band = out_data.GetRasterBand(i+1)
        out_band.WriteArray(values_to_write[i, :, :])
    out_band.FlushCache()
    out_band.SetNoDataValue(-32767.)

    out_data.SetGeoTransform(in_data.GetGeoTransform())
    out_data.SetProjection(in_data.GetProjection())
    del out_data
    return raster_output

In [ ]:
path_Haralick = '/Forest/HaralickTask_all_part/'
path_Haralick_features = [path_Haralick+f+'/' for f in os.listdir(path_Haralick) if f!='tiff']
path_Haralick_npy = 'data/BANDS-S2-L2A_HARALICK.npy'

In [ ]:
path_terrain = '/Forest/Terrain/'

In [ ]:
input_raster = path_min_cloud
output_raster = path_Haralick+'tiff/'
if not os.path.exists(output_raster):
    os.mkdir(output_raster)

field = np.array(gdal.Open(path_min_cloud).ReadAsArray())
Haralick_npy_shape = np.load(path_Haralick_features[0]+'0/'+path_Haralick_npy)[0]
for path_Haralick_feature in path_Haralick_features:
    if not os.path.exists(output_raster+path_Haralick_feature.split(path_Haralick)[1].split('/')[0]+'.tiff'):
        array_reloaded = np.zeros(shape=(field.shape[1], field.shape[2], Haralick_npy_shape.shape[2]))
        for i in range(4):
            array_reloaded[Haralick_npy_shape.shape[0]*(1-i%2):Haralick_npy_shape.shape[0]*(2-i%2),Haralick_npy_shape.shape[1]*(i//2):Haralick_npy_shape.shape[1]*(i//2+1),:] = np.load(path_Haralick_feature+str(i)+'/'+path_Haralick_npy)[0]
        res_fname = convert_to_geotiff(raster_input = input_raster, raster_output=output_raster+path_Haralick_feature.split(path_Haralick)[1].split('/')[0]+'.tiff', values_to_write=array_reloaded.transpose(2, 0, 1))
        print(res_fname)

In [ ]:
path_Haralick_tiffs = [output_raster+f for f in os.listdir(output_raster)]

In [ ]:
def generate_NDVI(features):
    nir = features[7]
    red = features[3]
    ndvi = (nir-red)/((nir+red).apply(lambda x: 0.000000001 if x==0 else x))
    return ndvi

In [ ]:
def generate_EVI(features):
    evi2 = 2.5*(features[7] - features[3])/((features[7]  + 2.4*features[3] + 1).apply(lambda x: 0.000000001 if x==0 else x))
    return evi2

In [ ]:
def generate_NDRE(features):
    ndre = (features[7] - features[4])/((features[7] + features[4]).apply(lambda x: 0.000000001 if x==0 else x))
    return ndre

In [ ]:
def generate_MSAVI(features):
    nir = features[7]
    red = features[3]
    msavi=(2*nir + 1 - ((2*nir+1)**2 - 8*(nir-red))**(1/2))/2
    return msavi

In [ ]:
def generate_all_indices(df_def):
    df_def_col = list(df_def.columns)
    df_def['ndvi'] = generate_NDVI(df_def)
    df_def['evi'] = generate_EVI(df_def)
    df_def['ndre'] = generate_NDRE(df_def)
    df_def['msavi'] = generate_MSAVI(df_def)
    df_def = df_def.reindex(columns=df_def_col[:-2]+list(df_def.columns[-4:])+df_def_col[-2:])
    return df_def

In [ ]:
def del_row(df_def, create_id_test=False):
    del_row_col = []
    for col in df_def.columns[:-1]:
        if not create_id_test:
            if (col[0]=='2' or col[0]=='3') and col[2]=='2':
                del_row_col.append(col)
        else:
            if str(col)[0]=='2' or str(col)[0]=='3':
                del_row_col.append(col)
    del_row_cond = (df_def[del_row_col[0]]>0)
    for col in del_row_col[1:]:
        del_row_cond = del_row_cond&(df_def[col]>0)

    df_def = df_def.loc[del_row_cond].reset_index(drop=True)
    return df_def

In [ ]:
def get_mask(path_field_def, path_masks_def, create_train_test=True, split_test=0.8):
    with open(path_masks_def) as f:
        gj = geojson.load(f)
    features_masks = [feature["geometry"] for feature in gj['features']]
    features_ind = [feature["properties"]["t_Class"] for feature in gj['features']]
    features_ind = [i-3 if i>7 else i for i in features_ind]
    if not create_train_test:
        mask_data = 0
        with rasterio.open(path_field_def) as src:
            for i in np.unique(features_ind):
                indices_i = [feature_ind for feature_ind, x in enumerate(features_ind) if x == i]
                list_masks = [features_masks[index] for index in indices_i]
                out_image_full, out_transform = rasterio_mask(src, list_masks)
                out_image_full[out_image_full > 0] = i
                mask_data += out_image_full
        mask_data = mask_data[0,:,:]
        return mask_data
    else:
        mask_data_train = 0
        mask_data_test = 0
        with rasterio.open(path_field_def) as src:
            for i in np.unique(features_ind):
                indices_i = [feature_ind for feature_ind, x in enumerate(features_ind) if x == i]
                list_masks = [features_masks[index] for index in indices_i]
                if len(list_masks)>2:
                    patr_i = math.ceil(len(list_masks)*split_test)
                    random.shuffle(list_masks)
                    list_masks_train = list_masks[:patr_i]
                    list_masks_test = list_masks[patr_i:]
                    out_image_full, out_transform = rasterio_mask(src, list_masks_train)
                    out_image_full[out_image_full > 0] = i
                    mask_data_train += out_image_full
                    out_image_full, out_transform = rasterio_mask(src, list_masks_test)
                    out_image_full[out_image_full > 0] = i
                    mask_data_test += out_image_full
                else:
                    out_image_full, out_transform = rasterio_mask(src, list_masks)
                    out_image_full[out_image_full > 0] = i

                    mask_i = out_image_full[0].copy()
                    mask_i[mask_i > 0] = 1
                    mask_arange_i = np.arange(mask_i.shape[0]*mask_i.shape[1]).reshape(mask_i.shape)*mask_i.copy()
                    element_i = mask_arange_i.copy().reshape(mask_i.shape[0]*mask_i.shape[1])
                    element_i = element_i[element_i != 0]
                    element_i = element_i[math.ceil(len(element_i)*split_test)]

                    mask_data_train_i = np.zeros(mask_i.shape, dtype=np.uint16)
                    mask_data_test_i = np.zeros(mask_i.shape, dtype=np.uint16)
                    mask_data_train_i[mask_arange_i<=element_i] = 1
                    mask_data_test_i[mask_arange_i>element_i] = 1

                    mask_data_train += mask_data_train_i*out_image_full.copy()
                    mask_data_test += mask_data_test_i*out_image_full.copy()
        mask_data_train = mask_data_train[0,:,:]
        mask_data_test = mask_data_test[0,:,:]
        return mask_data_train, mask_data_test

In [ ]:
def get_pixels(a, shape_out):
    height = []
    width = []
    hw = (height, width)
    for i_step, i_shape in enumerate(a.shape[1:]):
        max_out = math.trunc(i_shape/shape_out)+1
        shift_i = (shape_out-i_shape%shape_out)//(i_shape//shape_out)
        shift_end = (shape_out-i_shape%shape_out)%(i_shape//shape_out)
        for i in range(max_out):
            if i!=max_out-1:
                hw[i_step].append((shape_out*i-shift_i*i, shape_out*(i+1)-shift_i*i))
            else:
                hw[i_step].append((shape_out*i-shift_i*i-shift_end, shape_out*(i+1)-shift_i*i-shift_end))
    return height, width

In [ ]:
def get_fields_coord(mask_field_def, path_field_def, shape_field, show_def = False):
    field_def = np.array(gdal.Open(path_field_def).ReadAsArray())
    fields_coord_def = []
    non_fields_coord_def = []
    height, width = get_pixels(field_def, 96)
    for field_coord in [list(x) for x in itertools.product(height, width)]:
        if mask_field_def[field_coord[0][0]:field_coord[0][1],field_coord[1][0]:field_coord[1][1]].sum()>0:
            fields_coord_def.append(field_coord)
        else:
            non_fields_coord_def.append(field_coord)
    if show_def:
        a_def = field_def[9,:,:].copy()
        for field_coord in fields_coord_def:
            a_def[field_coord[0][0]:field_coord[0][1],field_coord[1][0]:field_coord[1][1]] = field_def[9,field_coord[0][0]:field_coord[0][1],field_coord[1][0]:field_coord[1][1]]+5000
        figure, axis = plt.subplots(1, 2, figsize=(10,10))
        axis[0].axis("off")
        axis[0].imshow(a_def)
        a_def = field_def[9,:,:].copy()
        for field_coord in non_fields_coord_def:
            a_def[field_coord[0][0]:field_coord[0][1],field_coord[1][0]:field_coord[1][1]] = field_def[9,field_coord[0][0]:field_coord[0][1],field_coord[1][0]:field_coord[1][1]]+5000
        axis[1].axis("off")
        axis[1].imshow(a_def)
    return fields_coord_def, non_fields_coord_def

In [ ]:
class ForestDatasetPart(Dataset):

    def __init__(self, images_dir, mask_data_Dataset, fields_Dataset, tozero=False):
        self.mask_data_Dataset = mask_data_Dataset
        self.tozero = tozero
        self.images_fields = {}
        for i_dir, image_dir in enumerate(images_dir):
            for i_field, field_Dataset in enumerate(fields_Dataset):
                self.images_fields[i_dir*len(fields_Dataset)+i_field] = (image_dir, field_Dataset)

    def __len__(self):
        return len(self.images_fields)

    def __getitem__(self, idx):
        image = np.array(gdal.Open(self.images_fields[idx][0]).ReadAsArray())
        field_har = np.array(gdal.Open(path_Haralick_tiffs).ReadAsArray())
        field_ter = np.array(gdal.Open(path_terrain).ReadAsArray())
        image = np.concatenate((image, field_har), axis=0)
        image = np.concatenate((image, field_ter), axis=0)
        field_shape = self.images_fields[idx][1]
        image = image[:,field_shape[0][0]:field_shape[0][1],field_shape[1][0]:field_shape[1][1]]
        labels = self.mask_data_Dataset[field_shape[0][0]:field_shape[0][1],field_shape[1][0]:field_shape[1][1]]

        if self.tozero:
            labels_ones = labels.copy()
            labels_ones[labels_ones > 0] = 1
            image = image*labels_ones

        image = torch.from_numpy(image.astype(float)).float()
        labels = torch.tensor(labels.astype(float)).to(torch.long)
        return image, labels

In [ ]:
class UNet(nn.Module):
    def __init__(self,
                 n_channels,
                 num_classes,
                 min_channels=32,
                 max_channels=512, 
                 num_down_blocks=4):
        super(UNet, self).__init__()
        self.num_classes = num_classes
        self.n_channels = n_channels
        self.num_down_blocks = num_down_blocks
        
        # encoder
        self.enc_conv1 = nn.Sequential(
                                    nn.Conv2d(in_channels=self.n_channels, out_channels=56, kernel_size=3, padding=1),
                                    nn.BatchNorm2d(56),
                                    nn.ReLU(),
                                    nn.Conv2d(in_channels=56, out_channels=112, kernel_size=3, padding=1),
                                    nn.BatchNorm2d(112),
                                    nn.ReLU()
        )
        
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        
        self.enc_conv2 = nn.Sequential(
                                    nn.Conv2d(in_channels=112, out_channels=224, kernel_size=3, padding=1),
                                    nn.BatchNorm2d(224),
                                    nn.ReLU(),
                                    nn.Conv2d(in_channels=224, out_channels=224, kernel_size=3, padding=1),
                                    nn.BatchNorm2d(224),
                                    nn.ReLU()
        )
        
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        
        self.enc_conv3 = nn.Sequential(
                                    nn.Conv2d(in_channels=224, out_channels=448, kernel_size=3, padding=1),
                                    nn.BatchNorm2d(448),
                                    nn.ReLU(),
                                    nn.Conv2d(in_channels=448, out_channels=448, kernel_size=3, padding=1),
                                    nn.BatchNorm2d(448),
                                    nn.ReLU()
        )
        
        self.pool3 = nn.MaxPool2d(kernel_size=2)
        
        # bottleneck
        self.bottleneck_conv = nn.Sequential(
                                    nn.Conv2d(in_channels=448, out_channels=448, kernel_size=3, padding=1),
                                    nn.BatchNorm2d(448),
                                    nn.ReLU(),
                                    nn.Conv2d(in_channels=448, out_channels=448, kernel_size=3, padding=1),
                                    nn.BatchNorm2d(448),
                                    nn.ReLU()
        )
        
        # decoder
        self.upsample1 = nn.ConvTranspose2d(in_channels=448, out_channels=448, kernel_size=3, stride=2, padding=1,output_padding=1)
        
        self.dec_conv1 = nn.Sequential(
                                    nn.Dropout(p=0.5),
                                    nn.Conv2d(in_channels=896, out_channels=224, kernel_size=3, padding=1),
                                    nn.BatchNorm2d(224),
                                    nn.ReLU(),
                                    nn.Conv2d(in_channels=224, out_channels=224, kernel_size=3, padding=1),
                                    nn.BatchNorm2d(224),
                                    nn.ReLU()
        )
        
        self.upsample2 = nn.ConvTranspose2d(in_channels=224, out_channels=224, kernel_size=3, stride=2, padding=1,output_padding=1)
        
        self.dec_conv2 = nn.Sequential(
                                    nn.Dropout(p=0.5),
                                    nn.Conv2d(in_channels=448, out_channels=112, kernel_size=3, padding=1),
                                    nn.BatchNorm2d(112),
                                    nn.ReLU(),
                                    nn.Conv2d(in_channels=112, out_channels=112, kernel_size=3, padding=1),
                                    nn.BatchNorm2d(112),
                                    nn.ReLU()
        )
        
        self.upsample3 = nn.ConvTranspose2d(in_channels=112, out_channels=112, kernel_size=3, stride=2, padding=1,output_padding=1)
        
        self.dec_conv3 = nn.Sequential(
                                    nn.Dropout(p=0.5),
                                    nn.Conv2d(in_channels=224, out_channels=112, kernel_size=3, padding=1),
                                    nn.BatchNorm2d(112),
                                    nn.ReLU(),
                                    nn.Conv2d(in_channels=112, out_channels=56, kernel_size=3, padding=1),
                                    nn.BatchNorm2d(56),
                                    nn.ReLU(),
                                    nn.Conv2d(in_channels=56, out_channels=self.num_classes, kernel_size=3, padding=1)
        )
    
    def forward(self, inputs):
        #print(inputs.shape)
        h, w = inputs.shape[2], inputs.shape[3]
        pow = 2**self.num_down_blocks
        h_near = pow * (h//pow)
        w_near = pow * (w//pow)
        inputs_near = F.interpolate(inputs, (h_near, w_near))
        #print(inputs_near.shape)
        # encoder
        e1 = self.enc_conv1(inputs_near)
        p1 = self.pool1(e1)
        e2 = self.enc_conv2(p1)
        p2 = self.pool2(e2)
        e3 = self.enc_conv3(p2)
        p3 = self.pool3(e3)

        # bottleneck
        b = self.bottleneck_conv(p3)

        # decoder
        u1 = self.upsample1(b)
        c1 = torch.cat([u1, e3], dim=1)
        d1 = self.dec_conv1(c1)
        u2 = self.upsample2(d1)
        c2 = torch.cat([u2, e2], dim=1)
        d2 = self.dec_conv2(c2)
        u3 = self.upsample3(d2)
        c3 = torch.cat([u3, e1], dim=1)
        d3 = self.dec_conv3(c3)
        logits = F.interpolate(d3, (h, w))

        assert logits.shape == (inputs.shape[0], self.num_classes, inputs.shape[2], inputs.shape[3]), 'Wrong shape of the logits'
        return logits

In [ ]:
class DeepLab(nn.Module):
    """
    Args:
      - backbone: ['resnet18', 'vgg11_bn', 'mobilenet_v3_small'],
      - aspp: use aspp module
      - num classes: num output classes

    During forward pass:
      - Pass inputs through the backbone to obtain features
      - Apply ASPP (if needed)
      - Apply head
      - Upsample logits back to the shape of the inputs
    """
    def __init__(self, backbone, aspp, num_classes):
        super(DeepLab, self).__init__()
        self.backbone = backbone
        self.init_backbone()
        self.num_classes = num_classes
        self.aspp_f = aspp

        if aspp:
            self.aspp = ASPP(self.out_features, 256, [12, 24, 36])

        self.head = DeepLabHead(self.out_features, num_classes)

    def init_backbone(self):
        # TODO: initialize an ImageNet-pretrained backbone
        if self.backbone == 'resnet18':
            resnet = models.resnet18(pretrained=True).children()
            self.resnet = torch.nn.Sequential(
                                    *(list(resnet)[:-2])
            )
            self.out_features = 512

        elif self.backbone == 'vgg11_bn':
            vgg = models.vgg11_bn(pretrained=True).features
            self.vgg = nn.Sequential(
                                    *list(vgg)
            )
            self.out_features = 512

        elif self.backbone == 'mobilenet_v3_small':
            mobilenet = models.mobilenet_v3_small(pretrained=True).features
            self.mobilenet = nn.Sequential(
                                    *list(mobilenet)
            )
            self.out_features = 576

    def _forward(self, x):
        # TODO: forward pass through the backbone
        if self.backbone == 'resnet18':
            x = self.resnet(x)

        elif self.backbone == 'vgg11_bn':
            x = self.vgg(x)

        elif self.backbone == 'mobilenet_v3_small':
            x = self.mobilenet(x)

        return x

    def forward(self, inputs):
        
        #Backbone
        inputs_i = self._forward(inputs)
        
        #ASPP
        if self.aspp_f:
            inputs_i = self.aspp.forward(inputs_i)
            
        #DeepLabHead
        inputs_i = self.head(inputs_i)
        
        #Upsample
        upsample = nn.Upsample(size = inputs.shape[2:], mode='bilinear')
        logits = upsample(inputs_i)

        assert logits.shape == (inputs.shape[0], self.num_classes, inputs.shape[2], inputs.shape[3]), 'Wrong shape of the logits'
        return logits


class DeepLabHead(nn.Sequential):
    def __init__(self, in_channels, num_classes):
        super(DeepLabHead, self).__init__(
            nn.Conv2d(in_channels, in_channels, 3, padding=1, bias=False),
            nn.BatchNorm2d(in_channels),
            nn.ReLU(),
            nn.Conv2d(in_channels, num_classes, 1)
        )


class ASPP(nn.Module):
    """
    Atrous Spatial Pyramid Pooling module
    with given atrous_rates and out_channels for each head
    Description: https://paperswithcode.com/method/aspp
    
    Detailed scheme: materials/deeplabv3.png
      - "Rates" are defined by atrous_rates
      - "Conv" denotes a Conv-BN-ReLU block
      - "Image pooling" denotes a global average pooling, followed by a 1x1 "conv" block and bilinear upsampling
      - The last layer of ASPP block should be Dropout with p = 0.5

    Args:
      - in_channels: number of input and output channels
      - num_channels: number of output channels in each intermediate "conv" block
      - atrous_rates: a list with dilation values
    """
    def __init__(self, in_channels, num_channels, atrous_rates):
        super(ASPP, self).__init__()
        
        modules = []
        
        modules.append(nn.Sequential(
                                nn.Conv2d(in_channels, num_channels, 1, 1),
                                nn.ReLU(),
                                nn.BatchNorm2d(num_channels)
        ))
        
        for rate in tuple(atrous_rates):
            modules.append(nn.Sequential(
                                nn.Conv2d(in_channels, num_channels, 3, 1, padding=rate, dilation=rate),
                                nn.BatchNorm2d(num_channels),
                                nn.ReLU()
            ))
        
        modules.append(nn.Sequential(
                                nn.AdaptiveAvgPool2d((1, 1)),
                                nn.Conv2d(in_channels, num_channels, 1, 1)
        ))
        
        self.convs = nn.ModuleList(modules)
    
        self.conv_out = nn.Conv2d(num_channels * len(self.convs), in_channels, 1, 1)
        
        self.drop = nn.Dropout(p = 0.5)
    
    def forward(self, x):
        blocks = []
        for conv in self.convs:
            blocks.append(conv(x))
        blocks[-1] = F.upsample(blocks[-1], size=x.shape[2:], mode='bilinear')
        
        res = torch.cat(blocks, dim=1)
        res = self.conv_out(res)
        res = self.drop(res)
        
        assert res.shape[1] == x.shape[1], 'Wrong number of output channels'
        assert res.shape[2] == x.shape[2] and res.shape[3] == x.shape[3], 'Wrong spatial size'
        return res

In [ ]:
def calc_val_data(preds, masks, num_classes):
    preds = torch.argmax(preds, dim=1)
        
    intersection = torch.zeros([masks.shape[0], num_classes])
    union = torch.zeros([masks.shape[0], num_classes])
    target = torch.zeros([masks.shape[0], num_classes])
    
    for i in range(num_classes):
        intersection[:,i] = torch.logical_and(preds == i, masks == i).sum(dim=(1,2))
        union[:,i] = torch.logical_or(preds == i, masks == i).sum(dim=(1,2))
        target[:,i] = (masks == i).sum(dim=(1,2))
    
    # Output shapes: B x num_classes

    assert isinstance(intersection, torch.Tensor), 'Output should be a tensor'
    assert isinstance(union, torch.Tensor), 'Output should be a tensor'
    assert isinstance(target, torch.Tensor), 'Output should be a tensor'

    assert intersection.shape == union.shape == target.shape, 'Wrong output shape'
    assert union.shape[0] == masks.shape[0] and union.shape[1] == num_classes, 'Wrong output shape'

    return intersection, union, target

def calc_val_loss(intersection, union, target, eps = 1e-7):
    mean_iou = torch.mean(intersection/(union+eps))
    mean_class_rec = torch.mean(intersection/(target+eps))
    mean_acc = torch.sum(intersection)/(torch.sum(target)+eps)

    return mean_iou, mean_class_rec, mean_acc

In [ ]:
class SegModel(pl.LightningModule):
    def __init__(
        self,
        model: str,
        backbone: str,
        aspp: bool,
        augment_data: bool,
        optimizer: str,
        scheduler: str,
        lr: float,
        batch_size: int,
        data_path: str,
        path_field_class: str,
        path_mask_class: str,
        image_size: int,
        num_classes: int,
        num_channels: int,
        tozero: bool
        ):
        super(SegModel, self).__init__()
        self.num_classes = num_classes

        if model == 'unet':
            self.net = UNet(num_channels, self.num_classes)
        elif model == 'deeplab':
            self.net = DeepLab(backbone, aspp, self.num_classes)

        mask_data_train, mask_data_test = get_mask(path_field_class, path_mask_class)
        fields_coord_train, _ = get_fields_coord(mask_data_train.copy(), path_field_class, image_size, False)
        fields_coord_test, _ = get_fields_coord(mask_data_test.copy(), path_field_class, image_size, False)
        self.train_dataset = ForestDatasetPart(data_path, mask_data_train.copy(), fields_coord_train, tozero=tozero)
        self.test_dataset = ForestDatasetPart(data_path, mask_data_test.copy(), fields_coord_test, tozero=tozero)



        self.batch_size = batch_size
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.lr = lr
        self.eps = 1e-7

        # Visualization
        self.color_map = torch.FloatTensor(
            [[0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1],
             [1, 0, 0], [1, 0, 1], [1, 1, 0], [1, 1, 1]])

    def forward(self, x):
        return self.net(x)

    def training_step(self, batch, batch_idx):
        img, mask = batch
        pred = self.forward(img)

        train_loss = F.cross_entropy(pred, mask)

        self.log('train_loss', train_loss, prog_bar=True)

        return train_loss

    def validation_step(self, batch, batch_idx):
        img, mask = batch
        pred = self.forward(img)

        val_loss = F.cross_entropy(pred, mask) 
        self.log('val_loss', val_loss, prog_bar=True)

        intersection, union, target = calc_val_data(pred, mask, self.num_classes)

        return {'intersection': intersection, 'union': union, 'target': target, 'img': img, 'pred': pred, 'mask': mask}

    def validation_epoch_end(self, outputs):
        intersection = torch.cat([x['intersection'] for x in outputs])
        union = torch.cat([x['union'] for x in outputs])
        target = torch.cat([x['target'] for x in outputs])

        mean_iou, mean_class_rec, mean_acc = calc_val_loss(intersection, union, target, self.eps)

        log_dict = {'mean_iou': mean_iou, 'mean_class_rec': mean_class_rec, 'mean_acc': mean_acc}

        for k, v in log_dict.items():
            self.log(k, v, prog_bar=True)

    def visualize_mask(self, mask):
        b, h, w = mask.shape
        mask_ = mask.view(-1)

        if self.color_map.device != mask.device:
            self.color_map = self.color_map.to(mask.device)

        mask_vis = self.color_map[mask_].view(b, h, w, 14).permute(0, 3, 1, 2).clone()

        return mask_vis

    def configure_optimizers(self):
        opt = torch.optim.Adam(self.net.parameters(), lr=self.lr, weight_decay=0.00001)
        sch = torch.optim.lr_scheduler.StepLR(opt, step_size=10, gamma=0.1)
        return [opt], [sch]

    def train_dataloader(self):
        return DataLoader(self.train_dataset, num_workers=2, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.test_dataset, num_workers=2, batch_size=1, shuffle=False)

In [ ]:
def define_model(model_name: str,
                 backbone: str,
                 aspp: bool,
                 augment_data: bool,
                 optimizer: str,
                 scheduler: str,
                 lr: float,
                 data_path: str,
                 path_field_class: str,
                 path_mask_class: str,
                 assignment_dir: str,
                 batch_size: int,
                 num_classes: int,
                 num_channels: int,
                 image_size: int,
                 tozero: bool,
                 checkpoint_name: str = ''):
    experiment_name = f'{model_name}_{backbone}_augment={augment_data}_aspp={aspp}_tozero={tozero}'
    model_name = model_name.lower()
    backbone = backbone.lower() if backbone is not None else backbone
    
    model = SegModel(
        model_name, 
        backbone, 
        aspp, 
        augment_data,
        optimizer,
        scheduler,
        lr,
        batch_size,
        data_path,
        path_field_class,
        path_mask_class,
        image_size,
        num_classes,
        num_channels,
        tozero)

    if checkpoint_name:
        model.load_state_dict(torch.load(f'{assignment_dir}/logs/{experiment_name}/{checkpoint_name}')['state_dict'])
    
    return model, experiment_name

def train(model, experiment_name, assignment_dir, use_gpu):
    logger = pl.loggers.TensorBoardLogger(save_dir=f'{assignment_dir}/logs', name=experiment_name)

    checkpoint_callback = pl.callbacks.ModelCheckpoint(
        monitor='mean_iou',
        dirpath=f'{assignment_dir}/logs/{experiment_name}',
        filename='{epoch:02d}-{mean_iou:.3f}',
        mode='max')
    
    trainer = pl.Trainer(
        max_epochs=100, 
        gpus=1 if use_gpu else None, 
        benchmark=True, 
        check_val_every_n_epoch=5, 
        logger=logger, 
        callbacks=[checkpoint_callback])

    time_start = time.time()
    
    trainer.fit(model)
    
    torch.cuda.synchronize()
    time_end = time.time()
    
    training_time = (time_end - time_start) / 60
    
    return training_time

In [ ]:
with open(path_masks) as f:
    gj = geojson.load(f)
features_ind = [feature["properties"]["t_Class"] for feature in gj['features']]

field = np.array(gdal.Open(path_min_cloud).ReadAsArray())

n_classes = len(np.unique(features_ind))
n_channels = field.shape[0]

assignment_dir = '/Forest/dl'

In [ ]:
model, experiment_name = define_model(
    model_name='UNet', #deeplab
    backbone=None,
    aspp=None,
    augment_data=False,
    optimizer='Adam',
    scheduler='StepLR',
    lr=0.001,
    data_path=path_snapshots,
    path_field_class=path_min_cloud,
    path_mask_class=path_masks,
    assignment_dir=assignment_dir,
    batch_size=16,
    num_classes=n_classes+1,
    num_channels=n_channels,
    image_size=96,
    tozero=True)

In [ ]:
training_time = train(model, experiment_name, assignment_dir, use_gpu=True)

print(f'Training time: {training_time:.3f} minutes')